In [18]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config_Sensor_1
settings = config_Sensor_1.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/karen/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [20]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config_Sensor_1
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [21]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-14 20:19:23,249 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-14 20:19:23,255 INFO: Initializing external client
2025-11-14 20:19:23,255 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-14 20:19:24,886 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279151


'{"country": "italy", "city": "rome", "street": "ViaClelia", "aqicn_url": "https://api.waqi.info/feed/A98194/", "latitude": 41.89, "longitude": 12.48}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [22]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_lagged_sensor1',
    version=3,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [23]:
import pandas as pd
from datetime import timedelta

three_days_ago = today - timedelta(days=3)

# Get the previous 3 days of pm25
lag = [
    util.get_pm25(aqicn_url, country, city, street, today - timedelta(days=d), AQICN_API_KEY)
    for d in range(1, 4)
]

# Extract pm25 values from each DataFrame (if not empty)
lag_pm25_values = [l['pm25'].values[0].astype('float32') for l in lag if not l.empty]

print("Lag values:", lag_pm25_values)

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)

# Add lag columns
for i, val in enumerate(lag_pm25_values, start=1):
    aq_today_df[f'lag_{i}'] = val

# Fill missing lag columns if fewer than 3 available
for i in range(len(lag_pm25_values) + 1, 4):
    aq_today_df[f'lag_{i}'] = None

aq_today_df

Lag values: [65.0, 65.0, 65.0]


,pm25,country,city,street,date,url,lag_1,lag_2,lag_3
0,65.0,italy,rome,ViaClelia,2025-11-14,https://api.waqi.info/feed/A98194/,65.0,65.0,65.0


In [24]:
# PM2.5 for today
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
 6   lag_1    1 non-null      float32       
 7   lag_2    1 non-null      float32       
 8   lag_3    1 non-null      float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 184.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [25]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df['street'] = street
daily_df

Coordinates 42.0°N 12.5°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,street
0,2025-11-14,17.850000,0.0,2.276840,71.564964,rome,ViaClelia
1,2025-11-15,21.299999,0.0,14.843180,165.963730,rome,ViaClelia
2,2025-11-16,21.049999,0.0,14.471821,174.289505,rome,ViaClelia
3,2025-11-17,19.549999,0.2,21.166653,228.447479,rome,ViaClelia
4,2025-11-18,17.299999,0.2,9.746631,184.236313,rome,ViaClelia
5,2025-11-19,12.600000,0.4,2.811690,50.194473,rome,ViaClelia
6,2025-11-20,16.549999,0.0,12.240000,208.072388,rome,ViaClelia


In [26]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
 6   street                       7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(2)
memory usage: 408.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-06 17:07:00,087 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279151/fs/1265760/fg/1638147


Uploading Dataframe: 100.00% |████████████████████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_lagged_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279151/jobs/named/air_quality_lagged_2_offline_fg_materialization/executions


(Job('air_quality_lagged_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735392
         }
       },
       "result": {
         "observed_value": 17.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-06T04:07:00.000086Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_e

In [10]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-06 17:07:14,019 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279151/fs/1265760/fg/1596082


Uploading Dataframe: 100.00% |████████████████████████████████████████████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279151/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-06 17:07:30,208 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-06 17:07:33,382 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-06 17:09:11,948 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-06 17:09:12,102 INFO: Waiting for log aggregation to finish.
2025-11-06 17:09:20,698 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 733254
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-06T04:07:14.000019Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
